In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Frame Bundle geometry on $\mathbb{S}^2$

In [ ]:
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *
M.plot()

In [ ]:
from src.Riemannian import metric
metric.initialize(M)

# element in FM (element in M, frame):
x = np.array([0.,0.])
u = np.array([[1.,-1.],[-1.,-1.]])
u = M.gramSchmidt(x,u)
q = np.hstack([x,u.flatten()])

# Covector in FM:
v = np.array([1.,0.])
px = M.flatf(x,v)
pu = np.array([1,0,1,0])
p = np.hstack([px,pu])

print("q = ", q)
print("p = ", p)

# Geodesic on Frame bundle:

In [ ]:
# Hamiltonian dynamics
from src.framebundle import FM
FM.initialize(M)
# test Hamiltionian and gradients
print(p)
print(M.Hfmf(q,p))

# geodesic
qsv = M.Exptfmf(q,p).T
#%matplotlib notebook
newfig()
M.plot(rotate=(30,-20))
M.plotFMx(qsv,N_vec=5,linewidth=1.5,s=50)
plt.show()
(ts,qpsv) = M.Hamfmf(q,p)
psv = qpsv[:,1,:]
print("Energy: ",np.array([M.Hfmf(q,p) for (q,p) in zip(qsv,psv)]))

In [ ]:
M.plotR2FMx(qsv,N_vec=5)
plt.show()

# Development and Stochastic Development

### Development

In [ ]:
x = np.array([0.1,0.1])

# Frame
u = np.array([[1,-1],[0,-1]]).astype(theano.config.floatX)
u = M.gramSchmidt(x,u)

Rot = np.array([[np.cos(45),np.sin(45)],[-np.sin(45),np.cos(45)]])
u = np.dot(Rot,u)
q = np.hstack([x,u.flatten()])
print("q = ", q)

# Curve in R^2:
t = np.linspace(0,10,n_steps.get_value()+1)
gamma = np.vstack([20*np.sin(t), t**2 + 2*t]).T
dgamma = np.diff(gamma, axis = 0)
print("dgamma.shape =", dgamma.shape)

In [ ]:
# Development dynamics
from src.stochastics import stochastic_development
stochastic_development.initialize(M)
# test deterministic development
qsv = M.devf(q,dgamma)
newfig()
M.plot(rotate=(30,-45))
M.plotFMx(qsv,linewidth=1.5,s=50)
plt.show()
#plt.savefig("development.pdf")

### Stochastic Development

In [ ]:
n_steps.set_value(1000)
x = np.array([0.,0.]).astype(theano.config.floatX)

u = np.array([[1.,-1],[1,1]]).astype(theano.config.floatX)
u = M.gramSchmidt(x,u)
print("x = ", x)
print("ui = ", u)

q = np.hstack([x.flatten(),u.flatten()]).astype(theano.config.floatX)
print("q = ", q)

dWt = np.random.normal(0, np.sqrt(dt.eval()), (n_steps.get_value(),2))
drift = np.array([0.5,0.5])

In [ ]:
# test stochastic development
qsv = M.stoc_devf(q,.8*dWt,drift)
newfig()
M.plot(rotate=(30,-45))
M.plotFMx(qsv,linewidth=1.5,s=50)
plt.show()
#plt.savefig("stocdev.pdf")

In [ ]:
Wt = np.cumsum(dWt,axis=0)
plt.figure()
plt.plot(Wt[:,0],Wt[:,1],'b-')
plt.plot(np.array([0]),np.array([0]),'go',markersize=5.)
plt.plot(Wt[-1:,0],Wt[-1:,1],'ro')
plt.show()
#plt.savefig("StocP.pdf")

# Curvature on FM

In [ ]:
import matplotlib.cm as cm
def plot_curvature(q,v0,v1,color='b'):

        x = q[0:d.eval()]
        ui = q[d.eval():].reshape((d.eval(),2))
        
        xq = x
        if x.shape[0] < 3: # map to S2
            x = Ff(x)
         
        ax = plt.gca(projection='3d')
        ax.scatter(x[0],x[1],x[2],color=color)

        # Frame along curve:
        curm = np.tensordot(np.tensordot(R_uif(xq,ui), v0, axes = [0,0]), v1, axes = [0,0])
        colors = cm.rainbow(np.linspace(0, 1, d.eval()))
        for j in range(d.eval()):
            JFgammai = JFf(xq)
            uiq = np.dot(JFgammai,ui[j,:])
            curV = np.dot(JFgammai,curm[j,:])
            ax.quiver(x[0],x[1],x[2],uiq[0],uiq[1],uiq[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=1.5,
                      color=colors[j],normalize=True,length=1.5*np.linalg.norm(uiq)/2)
            #end_Hvecq = (x + uiq/2)
            ax.quiver(x[0],x[1],x[2],
                      curV[0],curV[1],curV[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=2,
                      color=colors[j],normalize=True,length=np.linalg.norm(uiq)/2)

In [ ]:
# Curvature
from src.Riemannian import curvature
curvature.initialize(M)

cur = M.R_uif(x,ui)

v0 = np.array([1,0])
v1 = np.array([0,1])
curm = np.tensordot(np.tensordot(cur, ui0[:,0], axes = [0,0]), ui0[:,1], axes = [0,0])
print("curvature = ", curm)

# Plot of curvature:
newfig()
plotM(rotate=(50,-40))
plot_cuvature(q0,ui0[0,:],ui0[1,:])
plt.show()
#plt.savefig("Curvature.pdf")

# Most Probable Pahts

In [ ]:
# Hamiltonian dynamics
from src.FM import *

x1 = np.array([0,0])
x2 = np.array([0.5,-0.5])

v0 = np.array([[1./3,0.],[0.,1./3]]).astype(theano.config.floatX)
ui0 = GramSchmidt(v0,x1)
q1 = np.hstack([x1,ui0.flatten()]).astype(theano.config.floatX)
print("q = ", q1)

xi0 = np.array([1,0]).astype(theano.config.floatX)
xia = np.array([0,0,0,0]).astype(theano.config.floatX)

p0 = np.hstack([xi0,xia]).astype(theano.config.floatX)
print("p = ",p0)

In [ ]:
# Tangent vector giving the MPP:
Hor_v = Logfm(q1,x2)

# MPP from q1 to x2:
qsv = Exptfmf(q1,gMflatf(q1,Hor_v)).T

newfig()
ax = plt.gca(projection='3d')
plotM(rotate=(30,-20))
plotFMx(qsv,N_vec=5,linewidth=1.5,s=50)
xq = Ff(x2)
ax.scatter(xq[0],xq[1],xq[2],color='r')
plt.show()
#plt.savefig("MPP.pdf")

# Horizontal Vector Fields

In [ ]:
n_steps.set_value(100)
# Observation and frame
x0 = np.array([0.5,0.5]).astype(theano.config.floatX)

# Frame
v0 = np.array([[1,1],[0,-1]]).astype(theano.config.floatX)
ui0 = GramSchmidt(v0,x0)#sp.linalg.orth(v0) # Gramm-Schmidt!!

Rot = np.array([[np.cos(45),np.sin(45)],[-np.sin(45),np.cos(45)]])
ui0 = np.dot(Rot,ui0)
q0 = np.hstack([x0.flatten(),ui0.flatten()]).astype(theano.config.floatX)
print("q = ", q0)

t = np.linspace(0,1,n_steps.get_value())
gamma0 = 2*np.vstack([t**2+1,-t**2-np.sin(t)]).T
dgamma0 = 2*np.vstack([2*t,-2*t+np.cos(t)]).T

In [ ]:
def plotHorix(q,dgamma,i0=0,color='b',color_intensity=1.,linewidth=3.,prevx=None,last=True):
        if len(q.shape)>1:
            for i in range(q.shape[0]):
                plotHorix(q[i],dgamma,i0=i,# if prevx is None else None,
                      color=color,
                      linewidth=linewidth if i==0 or i==q.shape[0]-1 else .8,
                      color_intensity=color_intensity if i==0 or i==q.shape[0]-1 else .7,
                      prevx=q[i-1] if i>0 else None,
                      last=i==(q.shape[0]-1))
            return

        x = q[0:d.eval()]
        ui = q[d.eval():].reshape((d.eval(),2))
        
        xq = x
        if x.shape[0] < 3: # map to S2
            x = Ff(x)
         
        ax = plt.gca(projection='3d')
        if prevx is None or last:
            ax.scatter(x[0],x[1],x[2],color=color)
        if prevx is not None:
            prevxx = prevx[0:d.eval()]
            if prevxx.shape[0] < 3:
                prevxx = Ff(prevxx)
            xx = np.stack((prevxx,x))
            ax.plot(xx[:,0],xx[:,1],xx[:,2],linewidth=linewidth,color=color)

        # Frame along curve:
        if i0 is 0:
            Hvec = Horif(xq,ui)
            for j in range(d.eval()):
                JFgammai = JFf(xq)
                uiq = np.dot(JFgammai,ui[j,:])
                Hvecq = np.dot(JFgammai,Hvec[j,:])
                ax.quiver(x[0],x[1],x[2],uiq[0],uiq[1],uiq[2], pivot='tail',
                          arrow_length_ratio = 0.15, linewidths=1,
                          color='black',normalize=True,length=np.linalg.norm(uiq))
                ax.quiver(x[0],x[1],x[2],Hvecq[0],Hvecq[1],Hvecq[2], pivot='tail',
                          arrow_length_ratio = 0.15,normalize=True,linewidths=3, length=0.5,
                          color='red')
                #for k in range(rank.eval()):
                v = np.dot(Hvec[(2*j+2):(2*j+4),:],dgamma)
                Hvecui = np.dot(JFgammai,v)
                #Hvecui = np.dot(JFgammai,Hvec[(2*(j+1)+k),:])
                end_Hvecq = (x + uiq)
                ax.quiver(end_Hvecq[0],end_Hvecq[1],end_Hvecq[2],
                          Hvecui[0],Hvecui[1],Hvecui[2], pivot='tail',linewidths=2.,
                          normalize=True,length=0.3)
            # velocity vector:
            dgamma_v = np.dot(JFgammai,dgamma)
            ax.quiver(x[0],x[1],x[2],dgamma_v[0],dgamma_v[1],dgamma_v[2], pivot='tail',
                      arrow_length_ratio = 0.15,normalize=True,linewidths=3, length=0.7,
                      color='green')
                

In [ ]:
# Development dynamics
from src.Stochastic_Development import *
# test deterministic development
qsv = devf(q0,dgamma0)
Hori_v = Horif(x0,ui0)
print("Horizontal vector fields = ", Hori_v)

newfig()
plotM(rotate=(40,-45))
dqsv = qsv[1,0:d.eval()] - qsv[0,0:d.eval()]
plotHorix(qsv,dqsv)
#plt.savefig("Hori.pdf")

In [ ]:
print(Hori_v[d.eval():,:])
print(np.dot(Hori_v[d.eval():(d.eval()+2),:],dgamma0[0,:]))
print(np.dot(Hori_v[(d.eval()+2):(d.eval()+4),:],dgamma0[0,:]))

In [ ]:
print(DgMf(np.array([0.,0.])))